In [1]:
#-*- coding:utf-8 -*-'
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(pd.read_csv('DecisionTree.csv'))
map_Age = {'young':0,'middle-aged':1,'old':2}
map_Work = map_House = map_Label = {'no':0,'yes':1}
map_Loan = {'medium':0,'good':1,'excellent':2}
df['Age'] = df['Age'].map(map_Age)
df['Work'] = df['Work'].map(map_Work)
df['House'] = df['House'].map(map_House)
df['Loan'] = df['Loan'].map(map_Loan)
df['Label'] = df['Label'].map(map_Label)

data = df.iloc[:,1:-1].as_matrix()
label = df.iloc[:,-1].as_matrix()
label_cnt = len(map_Label)
feature_cnts = [len(map_Age), len(map_Work), len(map_House), len(map_Loan)]

In [3]:
def entropy(label):
    sum = 0
    for i in range(label_cnt):
        prop = len(label[label == i])/label.shape[0]
        if prop == 0:
            continue
        sum += prop*np.log2(prop)
    return -sum

In [4]:
def infomation_gain(data, label):
    H_D = entropy(label)
    H_D_A = np.array([0.0 for _ in range(len(feature_cnts))])
    for i in range(len(feature_cnts)):
        for j in range(feature_cnts[i]):
            H_D_ij = entropy(label[data[:,i] == j])
            H_D_A[i] += len(label[data[:,i] == j])*H_D_ij/label.shape[0]
    return H_D-H_D_A

In [5]:
infomation_gain(data, label)

array([0.0830075 , 0.3236502 , 0.41997309, 0.36298956])